In [1]:
from services import JamoService
from transformers import AutoTokenizer
import torch

In [4]:
tokenizer = AutoTokenizer.from_pretrained("hg_tokenizer")
jamo = JamoService("../../inference_server/model_store/production_A.tar")

In [5]:
del jamo

In [6]:
from jamo import JAMO

In [105]:
from pathlib import Path

model = JAMO.from_pretrained("small", "../../inference_server/model_store/production_A.tar", "cpu")

In [106]:
prompts = ["너는 누구니?", "안녕 반가워, 내 이름은 윤승현이라고 해.", "너가 가장 좋아하는 음식은?"]

def parsing_prmopt(instruction):
    chat_parser = (
        "명령어에 따른 요청을 적절히 완료하는 응답을 작성하세요.\n\n"
        "### 명령어:\n{instruction}\n\n### 응답:\n"
    )

    parsed_prompt = chat_parser.format_map({"instruction":instruction})
    return parsed_prompt
 
# idxs = torch.LongTensor([])
idxs = []
max_length = 0
for prompt in prompts:
    parsed_prmopt = parsing_prmopt(prompt)
    parsed_prmopt = f"<s> {parsed_prmopt}"
    
    idx = tokenizer.encode(parsed_prmopt)
    # idxs = torch.cat((idx, ))
    idxs.append(idx)
    if max_length < len(idx) : max_length = len(idx)

In [107]:
# def parse_for_multibatch(idx):
tmp = []
for idx in idxs:
    new = [1] * (max_length - len(idx)) + idx + [1] * (256-max_length)
    tmp.append(new)

target = torch.LongTensor(tmp)

In [108]:
target.shape

torch.Size([3, 256])

In [109]:
result = model(target)

In [110]:
result.shape

torch.Size([3, 256, 8000])

In [111]:
re = result[:, -1, :]
print(re.shape)
probs = torch.nn.functional.softmax(re, dim=-1)

probs.shape

torch.Size([3, 8000])


torch.Size([3, 8000])

In [112]:
next_token_idx = torch.multinomial(probs, num_samples=1)

In [113]:
next_token_idx


tensor([[1],
        [1],
        [1]])

In [114]:
next_token_idx.shape, target.shape

(torch.Size([3, 1]), torch.Size([3, 256]))

In [115]:
cur = max_length - 1

for i in range(10):
    print(i)
    result = model(target)

    re = result[:, cur, :]
    probs = torch.nn.functional.softmax(re, dim=-1)

    next_token_idx = torch.multinomial(probs, num_samples=1)
    cur += 1
    # print(target[:, cur].shape, next_token_idx.squeeze(1).shape)
    print(next_token_idx)
    
    target[:, cur] = next_token_idx.squeeze(1)

0
tensor([[723],
        [723],
        [723]])
1
tensor([[273],
        [273],
        [273]])
2
tensor([[414],
        [585],
        [397]])
3
tensor([[705],
        [380],
        [634]])
4
tensor([[886],
        [705],
        [800]])
5
tensor([[853],
        [384],
        [341]])
6
tensor([[ 373],
        [2307],
        [3744]])
7
tensor([[ 553],
        [1185],
        [3605]])
8
tensor([[490],
        [ 16],
        [341]])
9
tensor([[280],
        [608],
        [479]])


In [116]:
t = target[0].unsqueeze(0)

for i in range(10):
    result = model(t)

    re = result[:, cur, :]
    probs = torch.nn.functional.softmax(re, dim=-1)

    next_token_idx = torch.multinomial(probs, num_samples=1)
    cur += 1
    
    t[:, cur] = next_token_idx.squeeze(1)

In [118]:
empty = torch.empty((3, 10), dtype=torch.long)

In [127]:
idx = torch.randn((3, 5)).to(torch.long)

In [129]:
idx

tensor([[ 0,  0, -1,  0,  0],
        [ 0,  0,  0,  2, -1],
        [ 1, -1,  0,  1,  0]])

In [130]:
empty[range(3),: 5] = idx

In [131]:
empty

tensor([[ 0,  0, -1,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  2, -1,  0,  0,  0,  0,  0],
        [ 1, -1,  0,  1,  0,  0,  0,  0,  0,  0]])

In [132]:
pos = torch.arange(0, 10)
input_pos = torch.stack([pos, pos, pos], dim=0)

In [133]:
input_pos

tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])

In [136]:
input_pos[:,  -1:]+1

tensor([[10],
        [10],
        [10]])

In [137]:
pos[-1:]+1

tensor([10])

In [141]:
idx = torch.arange(5)
idx.index_copy(0, pos, 10)

IndexError: index_copy_(): Number of indices (10) should be equal to source.size(dim) (1)

In [169]:
idx_next = torch.tensor([1,3,0])

In [159]:
idx_next == 1

tensor([ True, False, False])

In [167]:
a = torch.zeros(3)

In [150]:
# a[idx_next==0] = 1

In [170]:
torch.where(idx_next!=0, a, 1.)

tensor([0., 0., 1.])

In [176]:
torch.any(torch.tensor([0, 0, 0]))

tensor(False)

In [182]:
idx = torch.ones((2, 5)).to(torch.long)
idx

tensor([[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1]])

In [186]:
idx.index_select(1, torch.arange(0, 3)).view(2, -1)

tensor([[1, 1, 1],
        [1, 1, 1]])

In [189]:
idx.index_copy(1, torch.arange(0, 3), torch.tensor([2, 3]))

IndexError: index_copy_(): When source and destination are not scalars, their dimensionality must match. Source dimensionality (1), destination dimensionality (2)

In [199]:
idx = torch.ones(4).to(torch.long)
idx.index_copy(0, torch.arange(0, 3), torch.tensor(10).to(torch.long))

IndexError: index_copy_(): When source is scalar, index should have one element (got 3)

In [200]:
torch.tensor([True, True, True]).tolist()

[True, True, True]

In [205]:
idx = torch.ones((2, 5))
print(idx)
idx[:, range(3)]

tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])


tensor([[1., 1., 1.],
        [1., 1., 1.]])

In [206]:
t = [[1, 2, 3], [71, 16, 2], [400, 50, 20]]

v, _ = torch.topk(torch.tensor(t), 1)

In [207]:
v

tensor([[  3],
        [ 71],
        [400]])

In [212]:
torch.where(torch.tensor(t) < v[:, [-1]], -float("Inf"), torch.tensor(t))

tensor([[-inf, -inf,   3.],
        [ 71., -inf, -inf],
        [400., -inf, -inf]])

In [214]:
torch.zeros(3)[torch.tensor([723, 723, 723])==2] = 1

In [227]:
torch.ones((2, 3), dtype=torch.long).index_copy(1, torch.tensor(2), torch.tensor([[0, 0, 4],[0, 0, 3]]))

IndexError: index_copy_(): Number of indices (1) should be equal to source.size(dim) (2)